In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Nov 14 09:55:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
import tensorflow as tf

In [ ]:
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

Cloning Repo

In [ ]:
# !git clone https://github.com/manishdhakal/ASR-Nepali-using-CNN-BiLSTM-ResNet

Cloning into 'ASR-Nepali-using-CNN-BiLSTM-ResNet'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 86 (delta 17), reused 80 (delta 11), pack-reused 0
Unpacking objects: 100% (86/86), done.


Downloading Dataset

In [ ]:
# %cd /content
# !mkdir ~/.kaggle 
# !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json 
# !chmod 600 ~/.kaggle/kaggle.json

/content


In [ ]:
# !kaggle datasets download anishshilpakar/asr-cnn-dataset

100% 6.88G/6.88G [06:12<00:00, 23.4MB/s]
100% 6.88G/6.88G [06:12<00:00, 19.9MB/s]


Extracting

In [ ]:
# %%capture
# %cd /content
# !unzip asr-cnn-dataset

In [ ]:
%%capture
%cd /content
!unzip /content/drive/MyDrive/audio.zip

In [ ]:
%%capture
!pip install edit_distance 
!pip install datasets
!pip install jiwer

In [ ]:
!python /content/drive/MyDrive/ASR-Nepali-using-CNN-BiLSTM-ResNet/trainer.py

2022-11-14 11:12:26.865577: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/content/drive/MyDrive/ASR-Nepali-using-CNN-BiLSTM-ResNet/trainer.py:19: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/device:GPU:0
Model defined ✅ ✅ ✅ ✅

Loading data.....
There are 22695 files
Data loaded ✅ ✅ ✅ ✅
And It took 28.128363132476807 seconds

Cleaning the audio files.....
Audio files cleaned ✅ ✅ ✅ ✅
And It took 18.67191505432129 seconds

Generating mfcc features.....
MFCC features generated ✅ ✅ ✅ ✅
And It took 528.8424408435822 seconds

Total Time for Loading Data: 575.6427190303802 seconds
Training epoch: 1
  9% 8/91 [05:50<52:48, 38.17s/it]

In [ ]:
!python /content/drive/MyDrive/ASR-Nepali-using-CNN-BiLSTM-ResNet/trainer.py

2022-11-14 07:02:30.576175: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/content/drive/MyDrive/ASR-Nepali-using-CNN-BiLSTM-ResNet/trainer.py:19: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/device:GPU:0
Model defined ✅ ✅ ✅ ✅

Loading data.....
10000 data loaded !!!
Data loaded ✅ ✅ ✅ ✅
And It took 58.04688501358032 seconds

Cleaning the audio files.....
Audio files cleaned ✅ ✅ ✅ ✅
And It took 19.06446361541748 seconds

Generating mfcc features.....


In [ ]:
!mkdir /content/drive/MyDrive/training_checkpoints

In [ ]:
# import json
# with open('/content/drive/MyDrive/Automatic-Nepali-Speech-Recognition-and-Summarizer/ASR/data_asr/vocabulary_asr/new_vocab.json','r',encoding='utf8') as f:
#     chars_dict = json.load(f)
# unq_chars = list(chars_dict.keys())
# unq_chars

In [ ]:
def calculateWER(actual_label, predicted_label):
    # convert string to list
    actual_words = actual_label.split()
    predicted_words = predicted_label.split()
    # costs will hold the costs like in Levenshtein distance algorithm
    costs = [[0 for inner in range(len(predicted_words)+1)] for outer in range(len(actual_words)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(predicted_words)+1)] for outer in range(len(actual_words)+1)]
    # ok means no change, sub means substitution, ins means insertion and del means deletion
    operations = {
        'ok': 0,
        'sub': 1,
        'ins': 2,
        'del': 3
    }
    # penalties for insertion, substitution and deletion
    penalties = {
        'ins': 1,
        'sub': 1,
        'del': 1
    }
    # First column represents the case where we achieve zero predicted labels i-e all the actual labels were deleted 
    for i in range(1,len(actual_words)+1):
        costs[i][0] = penalties['del']*i 
        backtrace[i][0] = operations['del']
    
    # First row represents the case where we achieve the predicted label by inserting all the predicted labels into a zero length actual label i-e all unwanted insertions 
    for j in range(1,len(predicted_words)+1):
        costs[0][j] = penalties['ins']*j 
        backtrace[0][j] = operations['ins']
    
    # computation
    for i in  range(1,len(actual_words)+1):
        for j in range(1,len(predicted_words)+1):
            # no change in predictions and actual label
            if actual_words[i-1] == predicted_words[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = operations['ok']
            else:
                # change has occured
                sub_cost = costs[i-1][j-1] + penalties['sub']
                ins_cost = costs[i][j-1] + penalties['ins']
                del_cost = costs[i-1][j] + penalties['del']
                costs[i][j] = min(sub_cost,ins_cost,del_cost)
                if costs[i][j] == sub_cost:
                    backtrace[i][j] = operations['sub']
                elif costs[i][j] == ins_cost:
                    backtrace[i][j] = operations['ins']
                else: 
                    backtrace[i][j] = operations['del']
    
    # backtrace through the best route
    i = len(actual_words)
    j = len(predicted_words)
    sub_count = 0 
    del_count = 0 
    ins_count = 0 
    correct_count = 0 

    while i > 0 or j > 0:
        if backtrace[i][j] == operations['ok']:
            correct_count += 1
            i -= 1
            j -= 1
        elif backtrace[i][j] == operations['sub']:
            sub_count += 1 
            i -= 1
            j -= 1
        elif backtrace[i][j] == operations['ins']:
            ins_count += 1
            j -= 1
        elif backtrace[i][j] == operations['del']:
            del_count += 1
            i -= 1
    
    """ 
    WER formula: 
    WER = S + D + I / N = S + D I / S + D + C
    """
    wer = round((sub_count + del_count + ins_count)/(sub_count + del_count + correct_count),3)
    # wer = round((sub_count + ins_count + del_count)/(float)(len(actual_words)),3)
    return wer 

In [ ]:
# Function to calculate the WER and CER 
def calculateErrorRates(actual_label,predicted_label):
    # For CER
    sm = ed.SequenceMatcher(predicted_label,actual_label)
    ed_dist = sm.distance() 
    cer = ed_dist/len(actual_label)
    # For WER 
    wer = calculateWER(actual_label,predicted_label)


In [ ]:
def calculateErrorRates(actual_label,predicted_label):
    # To load the evaluation metrics 
    wer_metric = load_metric("wer")
    cer_metric = load_metric("cer",revision="master")
    # Calculate CER and WER for given arguments 
    cer = cer_metric.compute(predictions=[predicted_label],references=[actual_label])
    wer = wer_metric.compute(predictions=[predicted_label],references=[actual_label])
    return cer,wer

In [ ]:
def calculateBatchErrorRates(output,target,start,end,cer,wer):
    """
        The line of codes below is for computing evaluation metric (CER) on internal validation data.
    """
    input_len = np.ones(output.shape[0]) * output.shape[1]
    # Decode the output using beam search and CTC to get  the required logits
    decoded_indices = K.ctc_decode(output, input_length=input_len,
                            greedy=False, beam_width=100)[0][0]
    
    # Remove the padding token from batchified target texts
    target_indices = [sent[sent != 0].tolist() for sent in target]

    # Remove the padding, unknown token, and blank token from predicted texts
    predicted_indices = [sent[sent > 1].numpy().tolist() for sent in decoded_indices] # idx 0: padding token, idx 1: unknown, idx -1: blank token

    batch_cer = cer
    batch_wer = wer
    len_batch = end - start
    for i in range(len_batch):
        predicted_label = predicted_indices[i]
        actual_label = target_indices[i]
        error_rates = calculateErrorRates(actual_label,predicted_label)
        batch_cer += error_rates[0]
        batch_wer += error_rates[1]
    batch_cer /= len_batch
    batch_wer /= len_batch
    return batch_cer,batch_wer

In [ ]:
def train_model(model, optimizer, train_wavs, train_texts, validation_wavs, validation_texts, epochs=100, batch_size=50):

    with tf.device(device_name):
        # These will be the final results to be returned
        train_losses = []
        validation_losses = []
        train_CERs = []
        train_WERs = []
        validation_CERs = []
        validation_WERs = []
        for e in range(0, epochs):
            start_time = time.time()
            len_train = len(train_wavs)
            len_validation = len(validation_wavs)
            train_loss = 0
            training_CER = 0
            training_WER = 0
            validation_loss = 0
            validation_CER = 0
            validation_WER = 0
            train_batch_count = 0
            validation_batch_count = 0
            # Training Steps
            print("Training epoch: {}".format(e+1))
            for start in tqdm(range(0, len_train, batch_size)):

                end = None
                if start + batch_size < len_train:
                    end = start + batch_size
                else:
                    end = len_train
                x, target, target_lengths, output_lengths = batchify(
                    train_wavs[start:end], train_texts[start:end], UNQ_CHARS)

                with tf.GradientTape() as tape:
                    output = model(x, training=True)

                    loss = K.ctc_batch_cost(
                        target, output, output_lengths, target_lengths)

                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))

                train_loss += np.average(loss.numpy())
                train_batch_count += 1
                training_cer, training_wer = calculateBatchErrorRates(output,target,start,end,training_CER,training_WER)


            # Validation Step
            print("Validation epoch: {}".format(e+1))
            for start in tqdm(range(0, len_validation, batch_size)):

                end = None
                if start + batch_size < len_validation:
                    end = start + batch_size
                else:
                    end = len_validation
                x, target, target_lengths, output_lengths = batchify(
                    validation_wavs[start:end], validation_texts[start:end], UNQ_CHARS)

                output = model(x, training=False)

                # Calculate CTC Loss
                loss = K.ctc_batch_cost(
                    target, output, output_lengths, target_lengths)

                validation_loss += np.average(loss.numpy())
                validation_batch_count += 1
                validation_cer, validation_wer = calculateBatchErrorRates(output,target,start,end,validation_CER,validation_WER)

            # Average the results
            # losses
            train_loss /= train_batch_count
            validation_loss /= validation_batch_count
            # cers
            train_CER /= train_batch_count
            validation_CER /= validation_batch_count
            # wers 
            train_WER /= train_batch_count 
            validation_WER /= validation_batch_count

            # Append the results 
            train_losses.append(train_loss)
            train_CERs.append(train_CER)
            train_WERs.append(train_WER)
            validation_losses.append(validation_loss)
            validation_CERs.append(validation_CER)
            validation_WERs.append(validation_WER)
            
            rec = f"Epoch: {e+1}, Train Loss: {train_loss:.2f}, Validation Loss: {validation_loss:.2f}, Train CER: {(train_CER*100):.2f}, Validation CER: {(validation_CER*100):.2f}, Train WER: {(train_WER*100):.2f}, Validation WER: {(validation_WER*100):.2f} in {(time.time() - start_time):.2f} secs\n"

            print(rec)

        result = {
            'epochs': range(0,epochs),
            'train_loss': train_losses,
            'validation_loss': validation_losses,
            'train_cer': train_CERs,
            'validation_cer': validation_CERs,
            'train_wer': train_WERs,
            'validation_wer': validation_WERs
        }
    
    return model, result

In [ ]:
# def train_model(model, optimizer, train_wavs, train_texts, test_wavs, test_texts, epochs=100, batch_size=50):

#     with tf.device(device_name):
#         train_losses = []
#         validation_losses = []
#         train_CERs = []
#         train_WERs = []
#         validation_CERs = []
#         validation_WERs = []
#         for e in range(0, epochs):
#             start_time = time.time()

#             len_train = len(train_wavs)
#             len_test = len(test_wavs)
#             train_loss = 0
#             test_loss = 0
#             test_CER = 0
#             train_batch_count = 0
#             test_batch_count = 0

#             print("Training epoch: {}".format(e+1))
#             for start in tqdm(range(0, len_train, batch_size)):

#                 end = None
#                 if start + batch_size < len_train:
#                     end = start + batch_size
#                 else:
#                     end = len_train
#                 x, target, target_lengths, output_lengths = batchify(
#                     train_wavs[start:end], train_texts[start:end], UNQ_CHARS)

#                 with tf.GradientTape() as tape:
#                     output = model(x, training=True)

#                     loss = K.ctc_batch_cost(
#                         target, output, output_lengths, target_lengths)

#                 grads = tape.gradient(loss, model.trainable_weights)
#                 optimizer.apply_gradients(zip(grads, model.trainable_weights))

#                 train_loss += np.average(loss.numpy())
#                 train_batch_count += 1

#             print("Testing epoch: {}".format(e+1))
#             for start in tqdm(range(0, len_test, batch_size)):

#                 end = None
#                 if start + batch_size < len_test:
#                     end = start + batch_size
#                 else:
#                     end = len_test
#                 x, target, target_lengths, output_lengths = batchify(
#                     test_wavs[start:end], test_texts[start:end], UNQ_CHARS)

#                 output = model(x, training=False)

#                 # Calculate CTC Loss
#                 loss = K.ctc_batch_cost(
#                     target, output, output_lengths, target_lengths)

#                 test_loss += np.average(loss.numpy())
#                 test_batch_count += 1

#                 """
#                     The line of codes below is for computing evaluation metric (CER) on internal validation data.
#                 """
#                 input_len = np.ones(output.shape[0]) * output.shape[1]
#                 decoded_indices = K.ctc_decode(output, input_length=input_len,
#                                        greedy=False, beam_width=100)[0][0]
                
#                 # Remove the padding token from batchified target texts
#                 target_indices = [sent[sent != 0].tolist() for sent in target]

#                 # Remove the padding, unknown token, and blank token from predicted texts
#                 predicted_indices = [sent[sent > 1].numpy().tolist() for sent in decoded_indices] # idx 0: padding token, idx 1: unknown, idx -1: blank token

#                 len_batch = end - start
#                 for i in range(len_batch):

#                     pred = predicted_indices[i]
#                     truth = target_indices[i]
#                     sm = ed.SequenceMatcher(pred, truth)
#                     ed_dist = sm.distance()                 # Edit distance
#                     test_CER += ed_dist / len(truth)
#                 test_CER /= len_batch

#             train_loss /= train_batch_count
#             test_loss /= test_batch_count
#             test_CER /= test_batch_count

#             rec = "Epoch: {}, Train Loss: {:.2f}, Test Loss {:.2f}, Test CER {:.2f} % in {:.2f} secs.\n".format(
#                 e+1, train_loss, test_loss, test_CER*100, time.time() - start_time)

#             print(rec)
#         return model

In [ ]:
import os 
len(os.listdir('/content/audio'))

142695

In [ ]:
!python /content/drive/MyDrive/ASR-Nepali-using-CNN-BiLSTM-ResNet/trainer.py

2022-11-13 16:59:00.181764: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/content/drive/MyDrive/ASR-Nepali-using-CNN-BiLSTM-ResNet/trainer.py:18: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
Model defined ✅ ✅ ✅ ✅

Loading data.....
Data loaded ✅ ✅ ✅ ✅

Cleaning the audio files.....
Audio files cleaned ✅ ✅ ✅ ✅

Generating mfcc features.....
MFCC features generated ✅ ✅ ✅ ✅

Training epoch: 1
100% 4/4 [00:10<00:00,  2.61s/it]
Validation epoch: 1
100% 1/1 [00:01<00:00,  1.31s/it]
Epoch: 1, Train Loss: 1280.79, Validation Loss: 1693.57, Train CER: 25.54, Validation CER: 82.85, Train WER: 41.78, Validation WER: 93.38 in 11.76 secs

Training epoch: 2
100% 4/4 [00:06<0

In [ ]:
import librosa
import pandas as pd 

In [ ]:
texts_df = pd.read_csv('/content/drive/MyDrive/Automatic-Nepali-Speech-Recognition-and-Summarizer/ASR/data_asr/transcript_asr/new_transcript_for_asr_complete.csv')[:10]
train_wavs = []
for f_name in texts_df["filename"]:
    wav, _ = librosa.load(f"/content/audio/audio/{f_name}.flac", sr=16000)
    train_wavs.append(wav)
train_texts = texts_df["label"].tolist()
print(train_wavs[-1],train_texts[3])

[ 2.9156721e-05  1.8241990e-05  3.8159305e-06 ... -8.6012187e-07
  7.8891588e-07 -6.7354171e-07] आन्तरिक राजस्व कार्यालय भैरहवा अन्तर्गत रुपन्देहीको सिद्धार्थनगर नगरपालिका लुम्बिनी नगरपालिका तिलोत्तमा नगरपालिकामा मूल्य अभिवृद्धि करमा दर्ता छ


In [ ]:
!python /content/drive/MyDrive/ASR-Nepali-using-CNN-BiLSTM-ResNet/eval.py

2022-11-11 16:33:13.369498: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Loading model.....
Model loaded ✅ ✅ ✅ ✅

Loading wav files.....
Wav files loaded ✅ ✅ ✅ ✅

Predicting sentences.....
['र्ोरनापछि चलचित्रको कथाको', 'जो इ स्लामको पहिलो'] 

Calculating CER.....
CER -> 13.29%, 	 No.of sentences -> 2, 	 Time Taken -> 2.15 secs.
The total time taken for all sentences CER calculation is  2.15 secs.
0.1328976034858388 

